In [1]:
#!pip install langchain
#!pip install chromadb
#!pip install tiktoken
#!pip install openai

import random
import os


from datetime import datetime, timedelta
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

# define global variable data_file
data_file = "schedule.txt"

In [2]:
# create some demo data
def create_data():
    # List of possible activities
    activities = [
        "Meeting with client",
        "Lunch break",
        "Project deadline",
        "Team brainstorming",
        "Training session",
        "Coffee break",
        "Presentation preparation",
        "Networking event",
        "Code review",
        "Project meeting"
    ]

    # List of possible names for meetings
    meeting_names = [
        "Emma",
        "Daniel",
        "Olivia",
        "Andrew",
        "Sophia"
    ]

    client_names = [
        "John",
        "Jane",
        "Michael",
        "Emily",
        "David"
    ]

    # Number of schedule entries to generate
    num_entries = 30

    # Generate random schedule entries with dates, times, and names for meetings
    schedule_entries = []
    start_date = datetime.now()

    for _ in range(num_entries):
        entry_date = start_date + timedelta(days=random.randint(0, 7))
        entry_time = random.randint(8, 17), random.randint(0, 59)
        activity = random.choice(activities)

        if activity == "Meeting with client":
            person = random.choice(client_names)
            schedule_entry = f"{entry_date.strftime('%Y-%m-%d')} {entry_time[0]:02d}:{entry_time[1]:02d} - {activity} with {person}"
        elif activity == "Project meeting":
            person = random.choice(meeting_names)
            schedule_entry = f"{entry_date.strftime('%Y-%m-%d')} {entry_time[0]:02d}:{entry_time[1]:02d} - {activity} with {person}"
        else:
            schedule_entry = f"{entry_date.strftime('%Y-%m-%d')} {entry_time[0]:02d}:{entry_time[1]:02d} - {activity}"

        schedule_entries.append(schedule_entry)

    # Write schedule entries to a text file
    with open(data_file, "w") as file:
        for entry in schedule_entries:
            file.write(entry + "\n")

    print(f"{num_entries} schedule entries have been written to schedule.txt.")


In [3]:
    create_data()
    load_dotenv()

    # Access the environment variable
    api_key = os.getenv("API_KEY")

    os.environ["OPENAI_API_KEY"] = api_key

    loader = TextLoader(data_file)
    index = VectorstoreIndexCreator().from_loaders([loader])

30 schedule entries have been written to schedule.txt.


In [4]:
    # test some queries on the index
    query = "Sum up the schedule"
    print(index.query(query))

    query = "Do I have Project meetings?"
    print(index.query(query))

    query = "What Meetings with clients do I have?"
    print(index.query(query))

    query = "Do I have Meetings with Olivia?"
    print(index.query(query))

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


 The schedule includes training sessions, project meetings, presentation preparation, team brainstorming, networking events, meetings with clients, coffee breaks, and lunch breaks.


Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


 Yes, you have Project meetings on 2023-07-03 08:05, 2023-07-03 12:49, and 2023-07-08 13:21.


Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


 You have meetings with clients with Jane, Michael, Emily, and Olivia.


Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


 No, you do not have a meeting with Olivia on any of the dates listed.
